In [3]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import h5py
from scipy import ndimage
from lr_utils import load_dataset

train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

## **Preprocessing**

### 1. Overview

m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print("Number of Training Examples : m_train = ", str(m_train))
print("Number of Test Examples : m_test = ", str(m_test))
print("Height/Width of each image : num_px = " , str(num_px))
print("Training_set_X shape = ", str(train_set_x_orig.shape))
print("Training_set_y shape = ", str(train_set_y.shape))
print("Test_set_X shape = ", str(test_set_x_orig.shape))
print("Test_set_y shape = ", str(test_set_y.shape))

### 2. Reshape

X_train = train_set_x_orig.reshape(m_train, -1).T
X_test = test_set_x_orig.reshape(m_test, -1).T

print("X_train shape = ", str(X_train.shape))
print("y_train shape = ", str(train_set_y.shape))
print("X_test shape = ", str(X_test.shape))
print("y_test shape = ", str(test_set_y.shape))

## 3. Rescale

## It is simpler and more convinient and works as well as standardization (z-score normalization) to just divide
# the rows by 255
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

## **Building Part of The cat Classifier**

def sigmoid(z):
    """
    args:
        z ndarray (1, m) -- Linear Part of the model
    return :
        a ndarray (1,m) -- prediction for each training Input
    """
    s = 1 / (1 + np.exp(-z))
    return s

def initialize_paramters(dim):
    """
    args : 
        dim scalar : number of features = width_image * height_image * depth_image
    return : 
        w ndarray (n , 1) : initialzed weights for each feature
        b float : bias
    """
    w = np.zeros((dim, 1))
    b = 0.0
    return w, b

def propagate (w, b, X, y):
    """
    args:
        w ndarray (n , 1) -- weights paramters of size (width_px * height_px * 3, 1)
        b scalar (float) -- bias
        X ndarray (n , m) -- Feature Vector x with (width_px * height_px * 3, 1)
        y ndarray (1, m) -- true labels for all training example
    returns :
    grads -- dictionary containing the gradients of weights and bias
        (dw -- gradient of the loss with respect to w , thus the same shape as w)
        (db -- gradient of the loss with respect to b , thus the same shape as b)
    cost -- negative log-likelood cost for logistic regression
    """
    m = X.shape[1]
    # compute cost 
    z = np.dot(w.T, X) + b
    a = sigmoid(z)
    loss = y * np.log(a) + (1 - y) * np.log(1 - a)
    cost = (-1/m) * np.sum(loss)

    # compute gradient
    dz = a - y
    dw = (1/m) * np.dot(X, dz.T)
    db = (1/m) * np.sum(dz)

    grads = {
        "dw" : dw,
        "db" : db
    }

    return grads, cost

def optimizer(w_init,b_init, X, y, iterations=100, learning_rate=0.009, print_cost=False,tolerance=1e-8):
    """
    Returns:
    params -- dictionary containing weights w & b bias
    grads  -- dictinary containing gradients of weights and bias with respect to cost function
    cost   -- list of all cost computed during the optimization - this will be used to plot learning curves
    """
    w = copy.deepcopy(w_init)
    b = copy.deepcopy(b_init)
    costs = []

    for i in range(iterations):
        grads , cost = propagate(w,b,X,y)
        dw = grads["dw"]
        db = grads["db"]

        w = w - learning_rate * dw
        b = b - learning_rate * db

        if i % 100 == 0:
            costs.append(cost)

        if print_cost:
            print(f"Iteration {i:5d} : Cost = {cost : 8.4f}")

        if len(costs) >= 2 and abs(costs[-1] - costs[-2]) < tolerance:
            print(f"Converges at {i : 5d}")
            break

    params = {"w":w,
            "b":b}
    grads = {"dw" : dw,
             "db" : db}
    
    return params, grads, cost
        

def predict (w, b , X):
    """
    args:
    w -- weights, a numpy array of size (lenth_px * height_px * 3, 1)
    b -- bias , a scalar (float)
    X -- Input Matrix, a Matrix of size (n , m)

    returns:
    Y_prediction -- Prediction (0 or 1), a numpy row vector of size (1 * m)
    """
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))

    A = sigmoid(np.dot(w.T, X) + b)

    for i in range(A.shape[1]):
        if A[0, i] >= 0.5:
            Y_prediction[0, i] = 1
        else:
            Y_prediction[0, i] = 0
    return Y_prediction

## **Define the Model**

def model (X_train, y_train, X_test, y_test, number_iteration=2000, learning_rate=0.5, print_cost=False):
    """
    returns:
    d -- information about the model (w, b, Y_prediction_train, Y_predition_test, cost, learning_rate, number_iteration)
    """
    w, b = initialize_paramters(X_train.shape[0])
    params, grads, costs = optimizer(w,b,X_train, y_train, number_iteration, learning_rate)
    w = params.get('w')
    b = params.get('b')
    Y_prediction_train = predict(w,b,X_train)
    Y_prediction_test = predict(w,b, X_test)

    if print_cost:
        print("Train Accuracy = {} %".format(100 - np.mean(np.abs(Y_prediction_train - y_train)) * 100 ))
        print("Test Accuracy = {} %".format(100 - np.mean(np.abs(Y_prediction_test - y_test)) * 100 ))


    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": number_iteration}
    
    return d

logistic_regression_model = model(X_train_scaled, train_set_y, X_test_scaled, test_set_y, number_iteration=2000, learning_rate=0.005, print_cost=True)

Number of Training Examples : m_train =  209
Number of Test Examples : m_test =  50
Height/Width of each image : num_px =  64
Training_set_X shape =  (209, 64, 64, 3)
Training_set_y shape =  (1, 209)
Test_set_X shape =  (50, 64, 64, 3)
Test_set_y shape =  (1, 50)
X_train shape =  (12288, 209)
y_train shape =  (1, 209)
X_test shape =  (12288, 50)
y_test shape =  (1, 50)
Train Accuracy = 99.04306220095694 %
Test Accuracy = 70.0 %
